In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import shutil

In [23]:
cascade = cv2.CascadeClassifier('cascade.xml') #分類器の指定
cap = cv2.VideoCapture('1.mp4')
senario_dict={}
i=0
if not os.path.isdir("./s_img"):
    os.mkdir("./s_img")

while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detectMultiScale(Mat image, MatOfRect objects, double scaleFactor, int minNeighbors, int flag, Size minSize, Size maxSize)
    detector = cascade.detectMultiScale(gray, 1.1, 3) #物体の検出
    if not detector:
        continue

    tmp1=gray.copy()
    tmp2=gray.copy()
    for (x, y, w, h) in detector:
        # タップライン
        height, width = img.shape[:2]
        tap_line=tmp1[y:y+h, 0:width]
        tap_line_name='./s_img/tap_line_{0}.jpg'.format(i)
        cv2.imwrite(tap_line_name, tap_line)
        senario_dict[i]['line']['base']=tmp1
        senario_dict[i]['line']['match']=tap_line
        senario_dict[i]['line']['is_parent']=True
        senario_dict[i]['line']['scale']=1
        
        # タップ位置
        tap_img=tmp2[y:y+h, x:x+w]
        tap_name='./s_img/tap_{0}.jpg'.format(i)
        cv2.imwrite(tap_name, tap_img)
        senario_dict[i]['tap']['base']=None
        senario_dict[i]['tap']['match']=tap_img
        senario_dict[i]['tap']['scale']=1
        
    i+=1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
f = open('{0}.json'.format("sample_senario"), 'w')
json.dump(senario_dict, f)

cap.release()
cv2.destroyAllWindows()